# Direct Fidelity Estimation of Quantum Processes

*Copyright (c) 2023 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline

Quantum Process Tomography gives complete information about the process we prepared, but it is very resource-consuming, and sometimes we only care about the fidelity between ideal and actual quantum process. Direct Fidelity Estimation (DFE) is a simple method for certifying that an experimental device prepares a desired quantum process.

This tutorial introduces Direct Fidelity Estimation (DFE) for state, covering its theory and implementation on [Baidu Quantum Platform](https://quantum.baidu.com/).

## Theory

### Fidelity estimation

The entanglement fidelity between our desired unitary channel $\mathcal{U}$ and the actual quantum channel $\mathcal{N}$ is given by

$$
    \mathcal{F}(\mathcal{U}, \mathcal{N}) = \frac{1}{d^2}\textrm{Tr}[\mathcal{M}_\mathcal{U}^\dagger \mathcal{M}_\mathcal{N}],
$$

where $\mathcal{M}_\mathcal{U}$ and $\mathcal{M}_\mathcal{N}$ are the corresponding matrix representations. We define $P_k(k=1,\cdots,d^2)$ as all possible Pauli operators, where $d=2^n$ and $n$ is the number of qubits. We know that both $\mathcal{U}$ and $\mathcal{N}$ can be expanded in Pauli basis, which is

$$
\begin{align}
    \textrm{Tr}[\mathcal{M}_\mathcal{U}^\dagger \mathcal{M}_\mathcal{N}]
    &= \frac{1}{d^2} \sum_{k, k'} \langle \langle P_k \vert \mathcal{U} \vert P_{k'} \rangle \rangle \langle \langle P_k \vert \mathcal{N} \vert P_{k'} \rangle \rangle \\
    &= \sum_{k,k'} \chi_{\mathcal{U}}(k,k') \chi_{\mathcal{N}}(k,k'),
\end{align}
$$

where $\chi_{\mathcal{U}}(k,k')$ is defined as

$$
\begin{align}
    \chi_{\mathcal{U}}(k,k')
    &= \langle \langle P_k \vert \mathcal{U} \vert P_{k'} \rangle \rangle \\
    &= \textrm{Tr}[P_k \mathcal{U}(P_k')]
\end{align}
$$

and $\chi_{\mathcal{N}(k,k')}$ is defined in a similar way.

Then we construct an estimator for entanglement fidelity as follows:

$$
\begin{align}
    \mathcal{F}(\mathcal{U}, \mathcal{N})
    &= \frac{1}{d^2} \sum_{k,k'} \chi_{\mathcal{U}}(k,k') \chi_{\mathcal{N}}(k,k') \\
    &= \sum_{k,k'}  \frac{\chi_{\mathcal{N}}(k,k')}{\chi_{\mathcal{U}}(k,k')} \frac{\chi_{\mathcal{U}}(k,k')^2}{d^2} \\
    &= \sum_{k,k'} X(k,k') \textrm{Pr}(k,k').
\end{align}
$$

Next, we describe the experimental process to perform the estimation.

Firstly, we sample Pauli pairs $l$ times according to probability distribution

$$
\begin{align}
    \textrm{Pr}(k,k')
    &= \frac{1}{d^2} \chi_{\mathcal{U}}(k,k')^2,
\end{align}
$$

which results in $((P_{k_1},P_{k'_1}), \cdots, (P_{k_l}, P_{k_l'}))$. Then for each pair of Pauli operators $(P_{k_i}, P_{k_i'})$, we estimate

$$
\begin{align}
    X(k_i, k_i')
    &= \frac{\chi_{\mathcal{N}}(k_i, k_i')}{\chi_{\mathcal{U}}(k_i, k_i')}
    &= \frac{\textrm{Tr}[P_{k_i} \mathcal{N}(P_{k_i'})]}{\textrm{Tr}[P_{k_i} \mathcal{U}(P_{k_i'})]}
\end{align}
$$

using the following procedure. We choose the eigenvalues of $P_{k_i'}$ uniformly, which is denoted as $\lambda_{a_{ij}} \in \left\{-1,1 \right\}$. Then, we prepare the corresponding eigenstate $\vert \psi_{a_{ij}} \rangle \langle \psi_{a_{ij}} \vert$ and apply $\mathcal{N}$ to the eigenstate. Next, we measure the Pauli observable $P_{k_i}$, and get the outcome $A_{ij} \in \{-1,1\}$. We define

$$
\begin{align}
B_{ij}
&= \lambda_{a_{ij}} A_{ij}.
\end{align}
$$

Accordingly, we have

$$
\begin{align}
    \mathbb{E}[B_{ij}]
    &= \mathbb{E}_{\lambda_{a_{ij}}}[\mathbb{E}[ \lambda_{a_{ij}} A_{ij} \vert \lambda_{a_{ij}}]] \\
    &= \mathbb{E}_{\lambda_{a_{ij}}}[\mathbb{E}[A_{ij}] \lambda_{a_{ij}} ] \\
    &= \sum_{a_{ij}=1}^d \frac{1}{d} \lambda_{a_{ij}} \mathbb{E}[A_{ij}] \\
    &= \textrm{Tr}(P_{k_i} \mathcal{N}(P_{k_i'})),
\end{align}
$$

which is equal to $\chi_{\mathcal{N}}(k_i, k_i')$. Therefore, we obtain

$$
\begin{align}
 \tilde{X}(k_i, k_i')
 &= \frac{\sum_{j=1}^{m_i} B_{ij}}{m_i \chi_{\mathcal{U}}(k_i, k_i')}.
\end{align}
$$

Finally, our estimator is

$$
\begin{align}
    \tilde{\mathcal{F}}(\mathcal{U}, \mathcal{N})
    &= \frac{1}{l} \sum_{i=1}^l \tilde{X}(k_i, k_i').
\end{align}
$$

### Parameter settings

There remains two questions:

1. How many pairs of Pauli operators do we need to sample?
2. For each pair of Pauli operators, how many measurements should we perform?

Below, we will address the above issues. Now we consider some fixed additional error $\epsilon$ and failure probability $\delta$. From Chebyshev's inequality and Hoeffding's inequality, we can use these two parameters [1] to calculate the sampling time $l$ and the number of shots $m_i$ for each Pauli operator pair at $i$.

We can conclude that with probability $\geq 1-2\delta$, the fidelity $\mathcal{\rho,\sigma}$ lies in $[\tilde{\mathcal{F}}(\mathcal{U } , \mathcal{N})-2\epsilon, \tilde{\mathcal{F}}(\mathcal{U}, \mathcal{N})+2\epsilon]$ , where $\tilde{\mathcal{ F }}(\mathcal{U}, \mathcal{N})$ is our estimate of the fidelity.


## Practice

We then demonstrate the process of using DFE to estimate the entanglement fidelity.

First, we import the necessary libraries.

In [ ]:
import QCompute
import Extensions.QuantumErrorProcessing.qcompute_qep.estimation as estimation

We then set up the quantum program for preparing Bell states on the 2nd and 3rd qubits.

In [ ]:
qp = QCompute.QEnv()  # qp is short for "quantum program", instance of QProgram
qp.Q.createList(3)
QCompute.H(qp.Q[1])
QCompute.CX(qp.Q[1], qp.Q[2])

![DFE](./figures/dfe-bell-state.png "Figure 1: We take the circuit of preparing Bell state as the target of DFE. ")

The circuit is shown in the figure above.

Then we set up the quantum computer (instance of QComputer). Quantum computer can be a simulator or a hardware interface. The rest is simple. We initialize a DFEProcess instance and call the estimate procedure. Here we set $\epsilon=\delta=0.05$ and choose the 2nd and 3rd qubits.

In [ ]:
# For numeric test, use the local ideal simulator
qc = QCompute.BackendName.LocalBaiduSim2

# Please log in the "Quantum Leaf" platform (https://quantum-hub.baidu.com/) to get Token
# QCompute.Define.hubToken = "Token"
# qc = QCompute.BackendName.CloudBaiduQPUQian

# Initialize a DFEProcess instance
est = estimation.DFEProcess()

# Call the estimate procedure
est.estimate(qp, qc, qubits=[1, 2], epsilon=0.05, delta=0.05)

Finally, we can obtain the fidelity between the ideal unitary channel and the actual process.

In [ ]:
print('Fidelity between the ideal and noisy states is: F = {:.5f}'.format(est.fidelity))

## References

[1] Flammia, Steven T., and Yi-Kai Liu. "Direct fidelity estimation from few Pauli measurements." **[Physical Review Letters](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.106.230501)** 106.23 (2011): 230501.

[2] da Silva, Marcus P., Olivier Landon-Cardinal, and David Poulin. "Practical characterization of quantum devices without tomography." **[Physical Review Letters](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.107.210404)** 107.21 (2011): 210404.

[3] Steffen, Lars, et al. "Experimental Monte Carlo quantum process certification." **[Physical Review Letters](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.108.260506)** 108.26 (2012): 260506.

[4] Kliesch, Martin, and Ingo Roth. "Theory of Quantum System Certification." **[PRX Quantum](https://journals.aps.org/prxquantum/abstract/10.1103/PRXQuantum.2.010201)** 2.1 (2021): 010201.